In [1]:
import polars as pl
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import json
from geopy.distance import great_circle
import openrouteservice
from openrouteservice import convert
import sys
sys.path.append('..')
import keys

In [2]:
df_data = pl.read_json(r'data.json')
df2 = pd.read_json(r'data.json')
df_old_data = pl.read_excel(r'Lojas Assaí.xlsx')

In [3]:
df_inaug22 = pl.read_csv('T2022.csv')
df_inaug23 = pl.read_csv('T2023.csv')
df_inaug24 = pl.read_csv('T2024.csv')
df_inaug_data = pl.concat([
    df_inaug24.rename({"column_0": "value"}),
    df_inaug23.rename({"column_0": "value"}),
    df_inaug22.rename({"column_0": "value"})
])


In [4]:
def start_pipeline(dataf):
    return dataf.clone()

def drop_columns_new(dataf):
    dataf = dataf.drop('url', 'subRegiao', 'subRegiaoTid', 'telefone', 'televendas', 'horario', 'email', 'ico_sust', 'voce_encontra', 'destaques', 'eslug', 'eid', 'e', 'tid', 'whatsapp', 'mapa', 'complemento', 'nid', 'cep', 'loja_id')
    return dataf

def drop_columns_old(dataf):
    dataf = dataf.drop('Unnamed: 4', 'Código Município')
    return dataf

def strip_chars_new(dataf):
    dataf = dataf.with_columns(pl.col("n").str.strip_chars())
    return dataf

def strip_chars_old(dataf):
    dataf = dataf.with_columns(pl.col("Unidade").str.strip_chars())
    return dataf

def rename_columns(dataf):
    dataf = dataf.rename({'n':'Unidade', 'c':'Município', 'uf':'UF', 'lat':'LAT', 'lon':'LONG', 'logradouro':'Endereço'})
    dataf = dataf.cast({"LAT": pl.Float64, "LONG": pl.Float64})
    return dataf

def sort_columns(dataf):
    dataf = dataf.select(['Unidade','Endereço','Município','UF','LAT','LONG'])
    return dataf

def format_names(dataf):
    dataf = dataf.with_columns(
        pl.col("value")
        .str.strip_chars()  # Remove trailing spaces
        .str.replace(r" - .*", "", literal=False)  # Remove anything after the "-"
        .str.to_lowercase()  # Convert to lowercase
    )
    return dataf

def filter_tri(dataf):
    dataf = dataf.filter(~pl.col("column_1").is_in(["1T2022", "2T2022"]))
    return dataf




In [5]:
df = (df_data
 .pipe(start_pipeline)
 .pipe(drop_columns_new)
 .pipe(strip_chars_new)
 .pipe(rename_columns)
 .pipe(sort_columns))

df_old = (df_old_data
 .pipe(start_pipeline)
 .pipe(strip_chars_old)
 .pipe(drop_columns_old))

df_inaug = (df_inaug_data
 .pipe(start_pipeline)
 .pipe(format_names)
 .pipe(filter_tri))

df_inaug

value,column_1
str,str
"""assaí vitória aeroporto""","""2T2024"""
"""assaí marginal tietê vila mari…","""1T2024"""
"""assaí santa rosa""","""1T2024"""
"""assaí zona norte""","""1T2024"""
"""assaí cidade tiradentes""","""1T2024"""
…,…
"""assaí guaianases""","""3T2022"""
"""assaí araraquara""","""3T2022"""
"""assaí cabula""","""3T2022"""


In [6]:

df_lower = df
df_old_lower = df_old
# Values in 'column_a' from df1 but not in 'column_b' from df2
only_in_df1 = df.filter(~pl.col("Unidade").str.to_lowercase().is_in(df_old['Unidade'].str.to_lowercase())).with_columns(pl.lit("Nova").alias("status"))

# Values in 'column_b' from df2 but not in 'column_a' from df1
only_in_df2 = df_old.filter(~pl.col('Unidade').str.to_lowercase().is_in(df["Unidade"].str.to_lowercase())).with_columns(pl.lit("Fechou").alias("status"))

# Common values in both dataframes with 'Antigas' status
common_in_both = df.filter(pl.col("Unidade").str.to_lowercase().is_in(df_old['Unidade'].str.to_lowercase())).with_columns(pl.lit("Antigas").alias("status"))

# Combine the results into one DataFrame
df_diff = pl.concat([
    only_in_df1.rename({"Unidade": "value"}),
    only_in_df2.rename({"Unidade": "value"}),
    common_in_both.rename({"Unidade": "value"})
])


df_diff = df_diff.with_columns(
    pl.col("value").str.to_lowercase().alias("value_lower")
)

# Cast 'value' in df_inaug to ensure compatibility
df_inaug = df_inaug.with_columns(pl.col("value").cast(pl.Utf8))

# Perform a left join to add `column_1` from df_inaug to df_diff based on the temporary lowercase column
df_diff = df_diff.join(df_inaug, left_on="value_lower", right_on="value", how="left")

# Update 'status' in df_diff if 'value_lower' is found in df_inaug (i.e., column_1 is not null)
df_diff = df_diff.with_columns(
    pl.when(pl.col("column_1").is_not_null())  # Check if 'column_1' exists after join
    .then(pl.lit("Nova"))  # Update status to 'Nova'
    .otherwise(pl.col("status"))  # Keep the original status otherwise
    .alias("status")
)

# Remove the temporary lowercase column after the join
df_diff = df_diff.drop("value_lower")
df_diff

value,Endereço,Município,UF,LAT,LONG,status,column_1
str,str,str,str,f64,f64,str,str
"""Assaí Zona Norte""","""Rua Tancredo Neves, 528""","""Macapá""","""AP""",0.06762,-51.05688,"""Nova""","""1T2024"""
"""Assaí Manaus Bola da Suframa""","""Rua Francisco Pereira da Silva…","""Manaus""","""AM""",-3.131969,-59.985704,"""Nova""",null
"""Assaí Salvador Paralela""","""Avenida Governador Luis Viana …","""Salvador""","""BA""",-12.96494,-38.43848,"""Nova""","""4T2023"""
"""Assaí Cais do Porto""","""Av. José Sabóia, 521""","""Fortaleza""","""CE""",-3.71718,-38.46671,"""Nova""","""4T2023"""
"""Assaí Montese""","""Avenida Dos Expedicionários, 4…","""Fortaleza""","""CE""",-3.753169,-38.537952,"""Nova""","""3T2023"""
…,…,…,…,…,…,…,…
"""Assaí Taboão da Serra""","""Rodovia Regis Bittencourt, 340""","""Taboão da Serra""","""SP""",-23.613221,-46.781066,"""Antigas""",null
"""Assaí Marginal Tietê Tatuapé""","""Rua Ulisses Cruz, nº 993""","""Tatuapé""","""SP""",-23.529796,-46.578482,"""Nova""","""4T2022"""
"""Assaí Taubaté""","""Avenida Dom Pedro I, 630 E,""","""Taubaté""","""SP""",-23.024431,-45.55644,"""Antigas""",null


In [27]:
# df.write_excel(r"lojasAssai.xlsx", worksheet='lojasSite')
# df_diff.write_excel(r"lojasAssaiDiff.xlsx", worksheet='lojasSite')


In [7]:
df_diff = pl.read_excel(r"lojasAssaiDiff_routetest.xlsx")
df_diff

value,Endereço,Município,UF,LAT,LONG,status,column_1
str,str,str,str,f64,f64,str,str
"""Assaí Galeão""","""Estrada do Galeão""","""Rio de Janeiro""","""RJ""",-22.804561,-43.206459,"""Conversao""","""1T2023"""
"""Assaí Raul Veiga""","""Estrada Raul Veiga""","""São Gonçalo""","""RJ""",-22.820851,-42.999547,"""Conversao""","""2T2023"""
"""Assaí Mogilar""","""Av Prefeito Carlos Ferreira Lo…","""Mogi das Cruzes""","""SP""",-23.512478,-46.188989,"""Nova""","""4T2022"""
"""Assaí Niterói Ponte""","""Rua Presidente Castelo Branco""","""Niterói""","""RJ""",-22.883342,-43.113063,"""Conversao""","""2T2023"""
"""Assaí Jaguaré Corifeu""","""Av Corifeu de Azevedo Marques,…","""São Paulo""","""SP""",-23.556875,-46.748359,"""Nova""","""4T2022"""
…,…,…,…,…,…,…,…
"""Assaí São Vicente""","""Avenida Antonio Emmerick, 245""","""São Vicente""","""SP""",-23.962595,-46.380576,"""Antigas""",null
"""Assaí Pirajussara""","""Avenida Ibirama, 51""","""Taboão da Serra""","""SP""",-23.64102,-46.8048,"""Antigas""",null
"""Assaí Taboão da Serra""","""Rodovia Regis Bittencourt, 340""","""Taboão da Serra""","""SP""",-23.613221,-46.781066,"""Antigas""",null


In [9]:
# ... (keep the existing code for data loading and preprocessing)

# After preprocessing, update the filtering and add color mapping:

# Define the categories we're interested in
categories_of_interest = ["Conversao", "Organica", "Nova", "Ex Combo Extra"]

# Filter for the categories of interest
new_stores_df = df_diff.filter(pl.col("status").is_in(categories_of_interest))
antigo_df = df_diff.filter(pl.col("status") == "Antigas")

# Create a color mapping for the categories
color_map = {
    "Conversao": "red",
    "Organica": "green",
    "Nova": "blue",
    "Ex Combo Extra": "purple",
    "Antigas": "grey"
}

# Create the OpenRouteService client
client = openrouteservice.Client(key=keys.API_OPENROUTESERVICE)

# Create lists for the new columns, initialized with None for all rows
closest_values_old = [None] * df_diff.height
closest_distances_old = [None] * df_diff.height
closest_values_old_LAT = [None] * df_diff.height
closest_values_old_LONG = [None] * df_diff.height
closest_values_all = [None] * df_diff.height
closest_distances_all = [None] * df_diff.height
route_geometries_old = [None] * df_diff.height
route_geometries_all = [None] * df_diff.height

# Iterate through each new store row
for index, row in enumerate(new_stores_df.iter_rows(named=True)):
    current_coords = (row["LAT"], row["LONG"])
    
    # Calculate distances to "Antigas" values
    distances_old = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in antigo_df.iter_rows(named=True)
    ]
    
    # Find the closest "Antigas" value
    if distances_old:
        closest_value_old, min_distance_old = min(distances_old, key=lambda x: x[1])
        closest_coords_old = next(row for row in antigo_df.iter_rows(named=True) if row["value"] == closest_value_old)
        closest_values_old[index] = closest_value_old
        closest_distances_old[index] = min_distance_old
        closest_values_old_LAT[index] = closest_coords_old["LAT"]
        closest_values_old_LONG[index] = closest_coords_old["LONG"]
    
    # Calculate distances to all other values
    distances_all = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in df_diff.iter_rows(named=True)
        if other_row["value"] != row["value"]  # Skip itself
    ]
    
    # Find the closest value among all
    if distances_all:
        closest_value_all, min_distance_all = min(distances_all, key=lambda x: x[1])
        closest_coords_all = next(row for row in df_diff.iter_rows(named=True) if row["value"] == closest_value_all)
        closest_values_all[index] = closest_value_all
        closest_distances_all[index] = min_distance_all
    
    # Calculate route for closest "Antigas" store
    if closest_values_old[index]:
        try:
            route_old = client.directions(
                [(row["LONG"], row["LAT"]), (closest_values_old_LONG[index], closest_values_old_LAT[index])],
                profile='driving-car',
                format='geojson'
            )
            route_geometries_old[index] = route_old['features'][0]['geometry']['coordinates']
        except Exception as e:
            print(f"Error calculating route for 'Antigas': {e}")
    
    # Calculate route for closest store among all
    if closest_values_all[index]:
        try:
            route_all = client.directions(
                [(row["LONG"], row["LAT"]), (closest_coords_all["LONG"], closest_coords_all["LAT"])],
                profile='driving-car',
                format='geojson'
            )
            route_geometries_all[index] = route_all['features'][0]['geometry']['coordinates']
        except Exception as e:
            print(f"Error calculating route for 'All': {e}")

# Add the new columns to the original DataFrame
df_diff = df_diff.with_columns([
    pl.Series("closest_value_old", closest_values_old),
    pl.Series("closest_distance_old", closest_distances_old),
    pl.Series("closest_value_old_LAT", closest_values_old_LAT),
    pl.Series("closest_value_old_LONG", closest_values_old_LONG),
    pl.Series("closest_value_all", closest_values_all),
    pl.Series("closest_distance_all", closest_distances_all),
    pl.Series("route_geometry_old", route_geometries_old),
    pl.Series("route_geometry_all", route_geometries_all)
])



c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 1st time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 2nd time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 3rd time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 4th time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 5th time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceed

In [10]:
# Plotting
df_diff = df_diff.with_columns(
    pl.lit(1).alias('dummy_column_for_size')
)

fig = px.scatter_mapbox(df_diff.to_pandas(), 
                        lat="LAT", 
                        lon="LONG", 
                        hover_name="value",
                        zoom=4,
                        height=600,
                        color='status',
                        color_discrete_map=color_map,
                        size='dummy_column_for_size',
                        size_max=10,)

fig.update_traces(
    hovertemplate=(
        '%{hovertext}<br>'
        'Status: %{marker.color}<br>'
        'Latitude: %{lat}<br>'
        'Longitude: %{lon}<br>'
    ),
    hovertext=df_diff['value'].to_pandas(),
)

lines = []
for row in new_stores_df.iter_rows(named=True):
    if row['closest_value_old_LAT'] is not None and row['closest_value_old_LONG'] is not None:
        lines.append(go.Scattermapbox(
            mode="lines",
            lon=[row['LONG'], row['closest_value_old_LONG']],
            lat=[row['LAT'], row['closest_value_old_LAT']],
            marker={'size': 1},
            line=dict(width=2, color=color_map[row['status']]),
            hoverinfo='none',
            showlegend=False
        ))

# Add all lines to the figure at once
fig.add_traces(lines)

route_lines = []
for row in new_stores_df.iter_rows(named=True):
    if row['route_geometry_old'] is not None:
        route_coords = row['route_geometry_old']
        lons, lats = zip(*route_coords)
        route_lines.append(go.Scattermapbox(
            mode="lines",
            lon=lons,
            lat=lats,
            line=dict(width=2, color=color_map[row['status']]),
            hoverinfo='none',
            showlegend=False
        ))

# Add all route lines to the figure
fig.add_traces(route_lines)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    autosize=True,
    height=1080,
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Convert the figure to JSON
plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# Prepare the data for search functionality
search_data = df_diff.select(['LAT', 'LONG', 'value', 'status']).to_dicts()
search_data_json = json.dumps(search_data)

# Create HTML file with search bar and JavaScript
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        #search-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 1000;
            background-color: white;
            padding: 10px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }}
        #search-input {{
            padding: 5px;
            width: 200px;
            margin-bottom: 5px;
        }}
        #search-results {{
            max-height: 200px;
            overflow-y: auto;
            border: 1px solid #ccc;
            display: none;
            background-color: white;
        }}
        .result-item {{
            padding: 5px;
            cursor: pointer;
        }}
        .result-item:hover {{
            background-color: #f0f0f0;
        }}
    </style>
</head>
<body>
    <div id="search-container">
        <input type="text" id="search-input" placeholder="Search for a point...">
        <div id="search-results"></div>
    </div>
    <div id="mapDiv"></div>

    <script>
        var plotlyData = {plot_json};
        var searchData = {search_data_json};
        
        Plotly.newPlot('mapDiv', plotlyData.data, plotlyData.layout);

        var searchInput = document.getElementById('search-input');
        var searchResults = document.getElementById('search-results');

        searchInput.addEventListener('input', function() {{
            var searchTerm = this.value.toLowerCase();
            
            var results = searchData.filter(function(point) {{
                return point.value.toLowerCase().includes(searchTerm);
            }});
            
            displayResults(results);
        }});

        function displayResults(results) {{
            searchResults.innerHTML = '';
            if (results.length > 0) {{
                results.forEach(function(result) {{
                    var div = document.createElement('div');
                    div.className = 'result-item';
                    div.textContent = result.value + ' (' + result.status + ')';
                    div.onclick = function() {{ 
                        centerOnPoint(result.LAT, result.LONG);
                        searchInput.value = result.value;
                        hideSearchResults();
                    }};
                    searchResults.appendChild(div);
                }});
                searchResults.style.display = 'block';
            }} else {{
                hideSearchResults();
            }}
        }}

        function centerOnPoint(lat, lon) {{
            Plotly.relayout('mapDiv', {{
                'mapbox.center': {{ lat: lat, lon: lon }},
                'mapbox.zoom': 15
            }});
        }}

        function hideSearchResults() {{
            searchResults.style.display = 'none';
        }}

        // Hide search results when clicking outside
        document.addEventListener('click', function(event) {{
            if (!searchContainer.contains(event.target)) {{
                hideSearchResults();
            }}
        }});

        // Hide search results when pressing Esc key
        document.addEventListener('keydown', function(event) {{
            if (event.key === 'Escape') {{
                hideSearchResults();
            }}
        }});

        // Prevent hiding when clicking inside the search container
        var searchContainer = document.getElementById('search-container');
        searchContainer.addEventListener('click', function(event) {{
            event.stopPropagation();
        }});
    </script>
</body>
</html>
"""

# Write the HTML content to a file
with open("localizacoesAssai_with_polars_search.html", "w") as f:
    f.write(html_content)

print("Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'")
print("Please open this file in a web browser and check the browser's console for debugging information.")

Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'
Please open this file in a web browser and check the browser's console for debugging information.


In [29]:
# Create the OpenRouteService client
client = openrouteservice.Client(key=keys.API_OPENROUTESERVICE)

# Create lists for the new columns, initialized with None for all rows
closest_values_old = [None] * df_diff.height
closest_distances_old = [None] * df_diff.height
closest_values_old_LAT = [None] * df_diff.height
closest_values_old_LONG = [None] * df_diff.height
closest_values_all = [None] * df_diff.height
closest_distances_all = [None] * df_diff.height
route_geometries_old = [None] * df_diff.height
route_geometries_all = [None] * df_diff.height

df_diff = df_diff.with_columns(
    (pl.col("status") == "Nova").alias("sort_order")  # Create a temporary column to sort by
)

df_diff = df_diff.sort(by="sort_order", descending=True)  # Sort by 'sort_order' column

# Drop the temporary sort column
df_diff = df_diff.drop("sort_order")

# Filter for "Antigo" values
nova_df = df_diff.filter(pl.col("status") == "Nova")
antigo_df = df_diff.filter(pl.col("status") == "Antigas")

# Iterate through each "Nova" row
for index, row in enumerate(nova_df.iter_rows(named=True)):
    current_coords = (row["LAT"], row["LONG"])
    
    # Calculate distances to "Antigas" values
    distances_old = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in antigo_df.iter_rows(named=True)
        if other_row["value"] != row["value"]  # Skip itself
    ]
    
    # Find the closest "Antigas" value
    if distances_old:
        closest_value_old, min_distance_old = min(distances_old, key=lambda x: x[1])
        closest_coords_old = next(row for row in antigo_df.iter_rows(named=True) if row["value"] == closest_value_old)
        closest_values_old[index] = closest_value_old
        closest_distances_old[index] = min_distance_old
        closest_values_old_LAT[index] = closest_coords_old["LAT"]
        closest_values_old_LONG[index] = closest_coords_old["LONG"]
    
    # Calculate distances to all other values
    distances_all = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in df_diff.iter_rows(named=True)
        if other_row["value"] != row["value"]  # Skip itself
    ]
    
    # Find the closest value among all
    if distances_all:
        closest_value_all, min_distance_all = min(distances_all, key=lambda x: x[1])
        closest_coords_all = next(row for row in df_diff.iter_rows(named=True) if row["value"] == closest_value_all)
        closest_values_all[index] = closest_value_all
        closest_distances_all[index] = min_distance_all
    
    # Calculate route for closest "Antigas" store
    if closest_values_old[index]:
        try:
            route_old = client.directions(
                [(row["LONG"], row["LAT"]), (closest_values_old_LONG[index], closest_values_old_LAT[index])],
                profile='driving-car',
                format='geojson'
            )
            route_geometries_old[index] = route_old['features'][0]['geometry']['coordinates']
        except Exception as e:
            print(f"Error calculating route for 'Antigas': {e}")
    
    # Calculate route for closest store among all
    if closest_values_all[index]:
        try:
            route_all = client.directions(
                [(row["LONG"], row["LAT"]), (closest_coords_all["LONG"], closest_coords_all["LAT"])],
                profile='driving-car',
                format='geojson'
            )
            route_geometries_all[index] = route_all['features'][0]['geometry']['coordinates']
        except Exception as e:
            print(f"Error calculating route for 'All': {e}")

# Add the new columns to the original DataFrame
df_diff = df_diff.with_columns([
    pl.Series("closest_value_old", closest_values_old),
    pl.Series("closest_distance_old", closest_distances_old),
    pl.Series("closest_value_old_LAT", closest_values_old_LAT),
    pl.Series("closest_value_old_LONG", closest_values_old_LONG),
    pl.Series("closest_value_all", closest_values_all),
    pl.Series("closest_distance_all", closest_distances_all),
    pl.Series("route_geometry_old", route_geometries_old),
    pl.Series("route_geometry_all", route_geometries_all)
])

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 1st time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 2nd time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 3rd time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 4th time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceeded. Retrying for the 5th time.

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning:

Rate limit exceed

In [33]:
# Plotting
df_diff = df_diff.with_columns(
    pl.lit(1).alias('dummy_column_for_size')
)

fig = px.scatter_mapbox(df_diff.to_pandas(), 
                        lat="LAT", 
                        lon="LONG", 
                        hover_name="value",
                        zoom=4,
                        height=600,
                        color='status',
                        size='dummy_column_for_size',
                        size_max=10,)

fig.update_traces(
    hovertemplate=(
        '%{hovertext}<br>'
    ),
    hovertext=df_diff['value'].to_pandas(),
)

lines = []
for row in df_diff.iter_rows(named=True):
    if row['closest_value_old_LAT'] is not None and row['closest_value_old_LONG'] is not None:
        if row['closest_distance_old'] < 5:
            line_color = 'red'
            line_width = 4
        elif row['closest_distance_old'] < 10:
            line_color = 'grey'
            line_width = 2
        else:
            line_color = 'grey'
            line_width = 2
        lines.append(go.Scattermapbox(
            mode="lines",
            lon=[row['LONG'], row['closest_value_old_LONG']],
            lat=[row['LAT'], row['closest_value_old_LAT']],
            marker={'size': 1},
            line=dict(width=line_width, color=line_color),
            hoverinfo='none',
            showlegend=False
        ))

# Add all lines to the figure at once
fig.add_traces(lines)

route_lines = []
for row in df_diff.iter_rows(named=True):
    if row['route_geometry_old'] is not None:
        route_coords = row['route_geometry_old']
        lons, lats = zip(*route_coords)
        route_lines.append(go.Scattermapbox(
            mode="lines",
            lon=lons,
            lat=lats,
            line=dict(width=2, color='blue'),
            hoverinfo='none',
            showlegend=False
        ))

# Add all route lines to the figure
fig.add_traces(route_lines)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    autosize=True,
    height=1080,
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Convert the figure to JSON
plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# Prepare the data for search functionality
search_data = df_diff.select(['LAT', 'LONG', 'value']).to_dicts()
search_data_json = json.dumps(search_data)

# Create HTML file with search bar and JavaScript
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        #search-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 1000;
            background-color: white;
            padding: 10px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }}
        #search-input {{
            padding: 5px;
            width: 200px;
            margin-bottom: 5px;
        }}
        #search-results {{
            max-height: 200px;
            overflow-y: auto;
            border: 1px solid #ccc;
            display: none;
            background-color: white;
        }}
        .result-item {{
            padding: 5px;
            cursor: pointer;
        }}
        .result-item:hover {{
            background-color: #f0f0f0;
        }}
    </style>
</head>
<body>
    <div id="search-container">
        <input type="text" id="search-input" placeholder="Search for a point...">
        <div id="search-results"></div>
    </div>
    <div id="mapDiv"></div>

    <script>
        var plotlyData = {plot_json};
        var searchData = {search_data_json};
        
        Plotly.newPlot('mapDiv', plotlyData.data, plotlyData.layout);

        var searchInput = document.getElementById('search-input');
        var searchResults = document.getElementById('search-results');

        searchInput.addEventListener('input', function() {{
            var searchTerm = this.value.toLowerCase();
            
            var results = searchData.filter(function(point) {{
                return point.value.toLowerCase().includes(searchTerm);
            }});
            
            displayResults(results);
        }});

        function displayResults(results) {{
            searchResults.innerHTML = '';
            if (results.length > 0) {{
                results.forEach(function(result) {{
                    var div = document.createElement('div');
                    div.className = 'result-item';
                    div.textContent = result.value;
                    div.onclick = function() {{ 
                        centerOnPoint(result.LAT, result.LONG);
                        searchInput.value = result.value;
                        hideSearchResults();
                    }};
                    searchResults.appendChild(div);
                }});
                searchResults.style.display = 'block';
            }} else {{
                hideSearchResults();
            }}
        }}

        function centerOnPoint(lat, lon) {{
            Plotly.relayout('mapDiv', {{
                'mapbox.center': {{ lat: lat, lon: lon }},
                'mapbox.zoom': 15
            }});
        }}

        function hideSearchResults() {{
            searchResults.style.display = 'none';
        }}

        // Hide search results when clicking outside
        document.addEventListener('click', function(event) {{
            if (!searchContainer.contains(event.target)) {{
                hideSearchResults();
            }}
        }});

        // Hide search results when pressing Esc key
        document.addEventListener('keydown', function(event) {{
            if (event.key === 'Escape') {{
                hideSearchResults();
            }}
        }});

        // Prevent hiding when clicking inside the search container
        var searchContainer = document.getElementById('search-container');
        searchContainer.addEventListener('click', function(event) {{
            event.stopPropagation();
        }});
    </script>
</body>
</html>
"""

# Write the HTML content to a file
with open("localizacoesAssai_with_polars_search.html", "w") as f:
    f.write(html_content)

print("Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'")
print("Please open this file in a web browser and check the browser's console for debugging information.")

Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'
Please open this file in a web browser and check the browser's console for debugging information.


In [31]:
# # Create the OpenRouteService client
# client = openrouteservice.Client(key=keys.API_OPENROUTESERVICE)

# # Create lists for the new columns, initialized with None for all rows
# closest_values_old = [None] * df_diff.height
# closest_distances_old = [None] * df_diff.height
# closest_values_old_LAT = [None] * df_diff.height
# closest_values_old_LONG = [None] * df_diff.height
# closest_values_all = [None] * df_diff.height
# closest_distances_all = [None] * df_diff.height
# route_geometries_old = [None] * df_diff.height

# df_diff = df_diff.with_columns(
#     (pl.col("status") == "Nova").alias("sort_order")  # Create a temporary column to sort by
# )

# df_diff = df_diff.sort(by="sort_order", descending=True)  # Sort by 'sort_order' column

# # Drop the temporary sort column
# df_diff = df_diff.drop("sort_order")

# # Filter for "Antigo" values
# nova_df = df_diff.filter(pl.col("status") == "Nova")
# antigo_df = df_diff.filter(pl.col("status") == "Antigas")

# # Iterate through each "Antigo" row
# for index, row in enumerate(nova_df.iter_rows(named=True)):
#     current_coords = (row["LAT"], row["LONG"])
#     # Calculate distances to other "Antigo" values
#     distances_old = [
#         (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
#         for other_row in antigo_df.iter_rows(named=True)
#         if other_row["value"] != row["value"]  # Skip itself
#     ]
#     # Find the closest "Antigo" value
#     if distances_old:
#         closest_value_old, min_distance_old = min(distances_old, key=lambda x: x[1])
#         closest_coords = next(row for row in antigo_df.iter_rows(named=True) if row["value"] == closest_value_old)
#         closest_values_old[index] = closest_value_old
#         closest_distances_old[index] = min_distance_old
#         closest_values_old_LAT[index] = closest_coords["LAT"]
#         closest_values_old_LONG[index] = closest_coords["LONG"]
#     # Calculate distances to other "Antigo" values
#     distances_all = [
#         (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
#         for other_row in df_diff.iter_rows(named=True)
#         if other_row["value"] != row["value"]  # Skip itself
#     ]
#     # Find the closest "Antigo" value
#     if distances_all:
#         closest_value_all, min_distance_all = min(distances_all, key=lambda x: x[1])
#         closest_values_all[index] = closest_value_all  # Set in the original position
#         closest_distances_all[index] = min_distance_all
#     # No need to append since we already initialized with None
#     try:
#         route = client.directions(
#             [current_coords, (closest_values_old_LONG[index], closest_values_old_LAT[index])],
#             profile='driving-car',
#             format='geojson'
#         )
#         distance = route['features'][0]['properties']['segments'][0]['distance'] / 1000  # Convert to km
#         route_geometries_old[index] = distance
#     except Exception as e:
#         print(f"Error calculating route: {e}")


# # Add the new columns to the original DataFrame
# df_diff = df_diff.with_columns([
#     pl.Series("closest_value_old", closest_values_old),
#     pl.Series("closest_distance_old", closest_distances_old),
#     pl.Series("closest_value_old_LAT", closest_values_old_LAT),
#     pl.Series("closest_value_old_LONG", closest_values_old_LONG),
#     pl.Series("closest_value_all", closest_values_all),
#     pl.Series("closest_distance_all", closest_distances_all),
#     pl.Series("route_old", route_geometries_old)
# ])



# # df_diff.write_excel("lojasAssaiMenorDistancia.xlsx", worksheet='lojasSite')



In [32]:
# # Assuming df_diff is your Polars dataframe
# df_diff = df_diff.with_columns(
#     pl.lit(1).alias('dummy_column_for_size')
# )

# fig = px.scatter_mapbox(df_diff.to_pandas(), 
#                         lat="LAT", 
#                         lon="LONG", 
#                         hover_name="value",
#                         zoom=4,
#                         height=600,
#                         color='status',
#                         size='dummy_column_for_size',
#                         size_max=10,)

# fig.update_traces(
#     hovertemplate=(
#         '%{hovertext}<br>'  # Customize hover text
#         # 'Latitude: %{lat}<br>'
#         # 'Longitude: %{lon}<br>'
#     ),
#     hovertext=df_diff['value'].to_pandas(),  # Use the 'value' column for hover text
#     )

# lines = []
# for row in df_diff.iter_rows(named=True):
#     # Ensure the closest coordinates are not None
#     if row['closest_value_old_LAT'] is not None and row['closest_value_old_LONG'] is not None:
#         if row['closest_distance_old'] < 5:
#             line_color = 'red'
#             line_width = 4
#         elif row['closest_distance_old'] < 10:
#             line_color = 'grey'
#             line_width = 2
#         else:
#             line_color = 'grey'
#             line_width = 2
#         lines.append(go.Scattermapbox(
#             mode="lines",
#             lon=[row['LONG'], row['closest_value_old_LONG']],
#             lat=[row['LAT'], row['closest_value_old_LAT']],
#             marker={'size': 1},
#             line=dict(width=line_width, color=line_color),
#             hoverinfo='none',
#             showlegend=False
#         ))

# # Add all lines to the figure at once
# fig.add_traces(lines)

# route_lines = []
# for row in df_diff.iter_rows(named=True):
#     if row['route_old'] is not None:
#         route_coords = row['route_old']
#         lons, lats = zip(*route_coords)
#         route_lines.append(go.Scattermapbox(
#             mode="lines",
#             lon=lons,
#             lat=lats,
#             line=dict(width=2, color='blue'),
#             hoverinfo='none',
#             showlegend=False
#         ))

# # Add all route lines to the figure
# fig.add_traces(route_lines)

# fig.update_layout(mapbox_style="carto-positron")
# fig.update_layout(
#     autosize=True,
#     height=1080,
#     margin={"r":0,"t":0,"l":0,"b":0}
# )


# # Convert the figure to JSON
# plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# # Prepare the data for search functionality
# search_data = df_diff.select(['LAT', 'LONG', 'value']).to_dicts()
# search_data_json = json.dumps(search_data)

# # Create HTML file with search bar and JavaScript
# html_content = f"""
# <!DOCTYPE html>
# <html>
# <head>
#     <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
#     <style>
#         #search-container {{
#             position: absolute;
#             top: 10px;
#             left: 10px;
#             z-index: 1000;
#             background-color: white;
#             padding: 10px;
#             border-radius: 5px;
#             box-shadow: 0 2px 5px rgba(0,0,0,0.2);
#         }}
#         #search-input {{
#             padding: 5px;
#             width: 200px;
#             margin-bottom: 5px;
#         }}
#         #search-results {{
#             max-height: 200px;
#             overflow-y: auto;
#             border: 1px solid #ccc;
#             display: none;
#             background-color: white;
#         }}
#         .result-item {{
#             padding: 5px;
#             cursor: pointer;
#         }}
#         .result-item:hover {{
#             background-color: #f0f0f0;
#         }}
#     </style>
# </head>
# <body>
#     <div id="search-container">
#         <input type="text" id="search-input" placeholder="Search for a point...">
#         <div id="search-results"></div>
#     </div>
#     <div id="mapDiv"></div>

#     <script>
#         var plotlyData = {plot_json};
#         var searchData = {search_data_json};
        
#         Plotly.newPlot('mapDiv', plotlyData.data, plotlyData.layout);

#         var searchInput = document.getElementById('search-input');
#         var searchResults = document.getElementById('search-results');

#         searchInput.addEventListener('input', function() {{
#             var searchTerm = this.value.toLowerCase();
            
#             var results = searchData.filter(function(point) {{
#                 return point.value.toLowerCase().includes(searchTerm);
#             }});
            
#             displayResults(results);
#         }});

#         function displayResults(results) {{
#             searchResults.innerHTML = '';
#             if (results.length > 0) {{
#                 results.forEach(function(result) {{
#                     var div = document.createElement('div');
#                     div.className = 'result-item';
#                     div.textContent = result.value;
#                     div.onclick = function() {{ 
#                         centerOnPoint(result.LAT, result.LONG);
#                         searchInput.value = result.value;
#                         hideSearchResults();
#                     }};
#                     searchResults.appendChild(div);
#                 }});
#                 searchResults.style.display = 'block';
#             }} else {{
#                 hideSearchResults();
#             }}
#         }}

#         function centerOnPoint(lat, lon) {{
#             Plotly.relayout('mapDiv', {{
#                 'mapbox.center': {{ lat: lat, lon: lon }},
#                 'mapbox.zoom': 15
#             }});
#         }}

#         function hideSearchResults() {{
#             searchResults.style.display = 'none';
#         }}

#         // Hide search results when clicking outside
#         document.addEventListener('click', function(event) {{
#             if (!searchContainer.contains(event.target)) {{
#                 hideSearchResults();
#             }}
#         }});

#         // Hide search results when pressing Esc key
#         document.addEventListener('keydown', function(event) {{
#             if (event.key === 'Escape') {{
#                 hideSearchResults();
#             }}
#         }});

#         // Prevent hiding when clicking inside the search container
#         var searchContainer = document.getElementById('search-container');
#         searchContainer.addEventListener('click', function(event) {{
#             event.stopPropagation();
#         }});
#     </script>
# </body>
# </html>
# """

# # Write the HTML content to a file
# with open("localizacoesAssai_with_polars_search.html", "w") as f:
#     f.write(html_content)

# print("Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'")
# print("Please open this file in a web browser and check the browser's console for debugging information.")